# trajectory_analyze

particle trajectory analysis

file requirements:
- particle netcdf file generated by parcels after simulation
- the actual ocean data the particles ran on

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

from utils import conv_to_dataarray, generate_mask_none, haversine, load_pts_mat
from parcels_utils import get_file_info
from plot_utils import plot_particles, get_carree_axis

In [ ]:
# ocean current netcdf data the particles used
d_nc = "current_netcdfs/west_coast_1km_hourly/tj_plume_interped.nc"
# the data generated by ParticleFile
p_nc = "particledata/particle_plume_track.nc"

# ocean current data
d_info = get_file_info(d_nc, 1)
d_ds = d_info["xrds"]
fs = d_info["fs"]

# particle data
p_ds = xr.open_dataset(p_nc)
p_ds.close()

In [ ]:
# lOLOLOLOLOLOL
# the good ol' click coordinates on google maps
coastline_points = np.array([
    [32.527430, -117.123895],
    [32.534549, -117.123832],
    [32.549035, -117.125920],
    [32.556846, -117.130490],
    [32.560474, -117.131934],
    [32.562632, -117.132144],
    [32.564511, -117.132933],
    [32.575317, -117.132941],
    [32.576679, -117.133227],
    [32.593312, -117.133008],
    [32.616745, -117.136719],
    [32.640275, -117.146353],
    [32.662540, -117.160633],
    [32.679572, -117.180263],
    [32.687357, -117.198503],
    [32.683428, -117.223487]
])

coastline_points = load_pts_mat("mat/coastline.mat", "latz0", "lonz0").T

tijuana_mouth = np.array([32.551967, -117.127208])

### to lat or to lon

coordinates are usually represnted as (lat, lon), but when doing maths and graphing and stuffs, it should be (lon, lat) since lon is on x-axis. the methods below do stuff according to (lon, lat) and reverse parameters accordingly.

In [ ]:
def line_seg(p1, p2, invert=True):
    if invert:
        p1 = (p1[1], p1[0])
        p2 = (p2[1], p2[0])
    return dict(
        p1=p1,
        p2=p2,
        dom=(p1[0], p2[0]) if p1[0] < p2[0] else (p2[0], p1[0]),
        rng=(p1[1], p2[1]) if p1[1] < p2[1] else (p2[1], p1[1]),
        slope=(p1[1] - p2[1]) / (p1[0] - p2[0])
    )


def valid_point(p, line, invert=True):
    if invert:
        p = (p[1], p[0])
    in_dom = p[0] >= line["dom"][0] and p[0] <= line["dom"][1]
    in_range = p[1] >= line["rng"][0] and p[1] <= line["rng"][1]
    return in_dom and in_range


def intersection_info(p, line, invert=True):
    """
    Returns:
        (intersection point), (haversine input)
    """
    if invert:
        p = (p[1], p[0])
    norm_slope = -1 / line["slope"]
    slope_d = norm_slope - line["slope"]
    int_d = (line["slope"] * -line["p1"][0] + line["p1"][1]) - (norm_slope * -p[0] + p[1])
    x_int = int_d / slope_d
    y_int = norm_slope * (x_int - p[0]) + p[1]
    return (x_int, y_int), (p[1], y_int, p[0], x_int)

In [ ]:
# line segment information for the coastline
coast_segs = np.empty(len(coastline_points) - 1, dtype=dict)
for i in range(0, len(coastline_points) - 1):
    coast_segs[i] = line_seg(coastline_points[i], coastline_points[i + 1])

In [ ]:
# line segment representation of coastline
ax = get_carree_axis(d_info["domain"])
plt.scatter(coastline_points.T[1], coastline_points.T[0], s=2)
plt.plot(coastline_points.T[1], coastline_points.T[0])

### distance to the coastline

get particles' closest distance to shore at every position and time saved in the particle data

In [ ]:
dists = np.empty((p_ds.dims["traj"], p_ds.dims["obs"]))
coastline_dense = True  # when there's enough points representing the coastline where segments are pointless
for i in range(dists.shape[0]):
    for j in range(dists.shape[1]):
        if np.isnan(p_ds["trajectory"].isel(traj=i, obs=j)):
            dists[i, j:] = np.nan
            break
        lat = float(p_ds["lat"].isel(traj=i, obs=j))
        lon = float(p_ds["lon"].isel(traj=i, obs=j))
        p = (lat, lon)
        least_dist = -1
        if not coastline_dense:
            # find closest coast line segment
            for seg in coast_segs:
                inted, hav_param = intersection_info(p, seg)
                valid = valid_point(inted, seg, invert=False)
                if valid:
                    dist = haversine(hav_param[0], hav_param[1], hav_param[2], hav_param[3])
                    if least_dist == -1 or dist < least_dist:
                        least_dist = dist
        # point is in a region not normal to any of the line segments
        # compare distances to defined points and find closest
        if least_dist == -1:
            for pnt in coastline_points:
                dist = haversine(p[0], pnt[0], p[1], pnt[1])
                if least_dist == -1 or dist < least_dist:
                    least_dist = dist
        dists[i, j] = least_dist

In [ ]:
d_mask = conv_to_dataarray(generate_mask_none(d_ds["u"].values), d_ds["u"])

In [ ]:
def time_to_str(time):
    """
    haha time go chop chop
    
    Args:
        time (np.datetime64)
    """
    return str(time).split(".")[0]

### check whether a particle collided with land

whether it's within the defined distance with the coastline (in meters)

OR

if the particle is in a coordinate without data and has 0 velocity

### some encountered problems

- particles are out of bounds even though I use coordinate values on the SAME FIELDSET during simulation
    - ??????????????? how


In [ ]:
collide_dist_thresh = 100  # meters
collided_land = np.zeros(p_ds["trajectory"].shape, dtype=bool)
lat_ls_coll = []
lon_ls_coll = []
times_sec = []
for p_iter in range(len(p_ds["trajectory"])):
    p = p_ds["trajectory"][p_iter]
    # find the first state of the particle where it got stuck on land something
    for step in range(len(p)):
        if np.isnan(p[step]):
            break
        time = p_ds["time"].values[p_iter, step]
        time_secs = p_ds["lifetime"].values[p_iter, step] + p_ds["spawntime"].values[p_iter, step]
        lat = p_ds["lat"].values[p_iter, step]
        lon = p_ds["lon"].values[p_iter, step]
        times_sec.append(time_secs)
        # check if the vector speed at that coordinate is 0 after interpolation
        # check if the particle is not in an invalid coordinate (a coordinate supposed to have data)
        total_vel = abs(fs.U[time_secs, 0, lat, lon]) + abs(fs.V[time_secs, 0, lat, lon])
        inland = total_vel <= 0 and d_mask.sel(time=time, lat=lat, lon=lon, method="nearest")
        # check if it's close enough to the coastline
        near_coast = dists[p_iter, step] <= collide_dist_thresh and dists[p_iter, step] >= 0
        if inland or near_coast:
            collided_land[p_iter, step] = True
            lat_ls_coll.append(lat)
            lon_ls_coll.append(lon)
            if inland:
                print(f"particle {p_iter} beached at time {time_to_str(time)} (inland)")
            else:
                print(f"particle {p_iter} beached at time {time_to_str(time)} (near coast)")
            print(f"    spawn time: {time_to_str(p_ds['time'].values[p_iter, 0])}")
            print(f"    spawn coord: ({p_ds['lat'].values[p_iter, 0]}, {p_ds['lon'].values[p_iter, 0]})")
            print(f"    lifetime: {p_ds['lifetime'].values[p_iter, step]}")
            # particle has hit coastline. we don't care about what happens after
            break

In [ ]:
# plot particles that have beached or something like that
plot_particles(lat_ls_coll, lon_ls_coll, None, d_info["domain"], land=False, show=False, part_size=5)
plt.plot(coastline_points.T[1], coastline_points.T[0])
plt.show()